In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [44]:
import itertools
import numpy as np
from scipy.optimize import linprog
from cvxopt import matrix, solvers, spmatrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from initialize import linparams

In [113]:
# Parameter values
n_class = 2
los = 3
capacity = 30
intensity = 0.9
slope_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [4]:
# intercepts and slopes for rate class, arrival day of week and los combination
params = linparams(capacity, intensity, slope_init, rates_init)

In [5]:
# Starting parameters
params_slopes = params[0]
params_intercepts = params[1]

In [6]:
params_slopes

array([[[-0.08122807, -0.07370269, -0.08375278],
        [-0.07544926, -0.08451997, -0.08142716],
        [-0.09322178, -0.07725568, -0.05773602],
        [-0.1068316 , -0.07412914, -0.06613756],
        [-0.08873431, -0.08650088, -0.07492901],
        [-0.09686144, -0.07815779, -0.06201683],
        [-0.0760522 , -0.07928701, -0.06180266]],

       [[-0.15483036, -0.13341666, -0.11434043],
        [-0.12637321, -0.09987329, -0.11079515],
        [-0.15404708, -0.11475417, -0.09260816],
        [-0.18590262, -0.09726223, -0.10104369],
        [-0.192081  , -0.13001256, -0.09647752],
        [-0.16013092, -0.11018703, -0.12105246],
        [-0.17224569, -0.08826453, -0.08778424]]])

In [7]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [8]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [27]:
obj_coefs = (-1) * rates_arrival_los.reshape(n_class * 7 * los)
type(obj_coefs)

numpy.ndarray

In [114]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combination.
C_ineq = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
C_ineq[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
C_ineq[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
C_ineq[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
C_ineq[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
C_ineq[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
C_ineq[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
C_ineq[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
C_ineq[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
C_ineq[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
C_ineq[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

# Inequality equations, RHS
# For each rate class, number of arrivals for a stay night in question is half of
# expected demand, which is capacity * intensity, then this expected demand is equally 
# split between 6 arrival day, los combination that spans the stay night in question
expDemand_each = (capacity * intensity * 0.5) / 6
d_ineq = expDemand_each * np.ones(n_class * 7 * los)
c_ineq = capacity * np.ones(7)
b_ineq = np.append(d_ineq, c_ineq)

In [127]:
C_ineq

array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.,

In [116]:
b_ineq

array([ 2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,
        2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,
        2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,
        2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,  2.25,
        2.25,  2.25,  2.25,  2.25,  2.25,  2.25, 30.  , 30.  , 30.  ,
       30.  , 30.  , 30.  , 30.  ])

In [117]:
C_ineq.shape

(7, 42)

In [118]:
D_ineq = np.identity(42, dtype=float)
A_ineq = np.concatenate((D_ineq, C_ineq), axis=0)

In [119]:
A_ineq.shape

(49, 42)

In [120]:
res = linprog(obj_coefs, A_ub=A_ineq, b_ub=b_ineq)

In [121]:
bkLimits = np.round(res['x'], decimals=0)

In [122]:
bkLimits

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2.])

In [123]:
print(res)

     con: array([], dtype=float64)
     fun: -22274.99999999035
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([1.00275344e-12, 4.53415083e-13, 4.79172257e-13, 2.18181029e-12,
       7.43405337e-13, 8.34443625e-13, 2.01261230e-12, 7.46958051e-13,
       4.82724971e-13, 2.18358664e-12, 4.48974191e-13, 5.40456568e-13,
       1.00275344e-12, 4.95603558e-13, 4.83169060e-13, 1.64757097e-12,
       5.37792033e-13, 4.80504525e-13, 1.64579461e-12, 4.94715380e-13,
       5.37792033e-13, 1.04449782e-12, 4.60076421e-13, 1.52278190e-12,
       3.09663406e-12, 6.75903777e-13, 3.02113889e-12, 2.26929586e-12,
       6.77680134e-13, 1.52011737e-12, 3.09485770e-12, 4.62740957e-13,
       9.85433957e-13, 1.04760645e-12, 5.47561996e-13, 5.28466160e-13,
       3.22852856e-12, 4.60964600e-13, 5.32462963e-13, 3.23563398e-12,
       5.44453371e-13, 9.98756633e-13, 3.00000000e+00, 3.00000000e+00,
       3.00000000e+00, 3.00000000e+00, 3.00000000e+00, 3.00000000e+00,
       3.000

In [124]:
b = matrix(b_ineq)
A = matrix(A_ineq)
c = matrix(obj_coefs, tc='d')

In [125]:
sol = solvers.lp(c, A, b)

     pcost       dcost       gap    pres   dres   k/t
 0: -2.4665e+04 -3.0332e+04  3e+03  1e-01  2e-01  1e+00
 1: -2.4445e+04 -2.8348e+04  2e+03  8e-02  2e-01  2e+01
 2: -2.3656e+04 -2.6384e+04  3e+03  6e-02  1e-01  3e+01
 3: -2.2425e+04 -2.2650e+04  5e+02  5e-03  1e-02  2e+01
 4: -2.2277e+04 -2.2281e+04  8e+00  7e-05  1e-04  3e-01
 5: -2.2275e+04 -2.2275e+04  8e-02  7e-07  1e-06  3e-03
 6: -2.2275e+04 -2.2275e+04  8e-04  7e-09  1e-08  3e-05
Optimal solution found.


In [126]:
print(sol['x'])

[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]
[ 2.25e+00]

